In [5]:
import tensorflow as tf

def read_my_file_format(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    
    record_defaults = [[1] for i in range(541)]
#     print(record_defaults)

    cols = tf.decode_csv(value, record_defaults=record_defaults)
    labels = tf.one_hot(cols[1], 2)
    features = tf.stack(cols[2:])

    return features, labels

def input_pipeline(filenames, batch_size, num_epochs=None):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs, shuffle=True)
    example, label = read_my_file_format(filename_queue)
  # min_after_dequeue defines how big a buffer we will randomly sample
  #   from -- bigger means better shuffling but slower start up and more
  #   memory used.
  # capacity must be larger than min_after_dequeue and the amount larger
  #   determines the maximum we will prefetch.  Recommendation:
  #   min_after_dequeue + (num_threads + a small safety margin) * batch_size
    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batch = tf.train.shuffle_batch(
        [example, label], batch_size=batch_size, capacity=capacity,
        min_after_dequeue=min_after_dequeue)
    return example_batch, label_batch


train_features_app, train_labels_app = input_pipeline(["data/train_approved.csv"], 100)
train_features_rej, train_labels_rej = input_pipeline(["data/train_rejected.csv"], 100)
train_features = tf.concat([train_features_app, train_features_rej], 0)
train_labels = tf.concat([train_labels_app, train_labels_rej], 0)

test_features, test_labels = input_pipeline(["data/test.csv"], 100)


init = tf.global_variables_initializer()

# with tf.Session() as sess:
#     sess.run(init)
    
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)

#     print( sess.run([train_features, train_labels]) )
    
#     coord.request_stop()
#     coord.join(threads)
    
# print("\nEnd.\n")


In [6]:

#define place holder
x = tf.placeholder(tf.float32, [None, 539])
y = tf.placeholder(tf.float32, [None, 2])

dense1 = tf.layers.dense(inputs=x, units=300, activation=tf.nn.relu)

prediction = tf.layers.dense(inputs=dense1, units=2, activation=tf.nn.softmax)

# #create a 1-level neural network
# W = tf.Variable(tf.truncated_normal([539, 2], stddev=0.1))
# # b = tf.Variable(tf.zeros([2]))
# b = tf.Variable(tf.zeros([2])+0.1)

# wx_b = tf.matmul(x, W)+b

#激活函数，实现非线性化
# prediction = tf.nn.relu(wx_b)
# prediction = tf.nn.tanh(wx_b)
# prediction = tf.nn.sigmoid(wx_b)
# prediction = tf.nn.softmax(wx_b)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
# loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=prediction))

#train
train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

init = tf.global_variables_initializer()

#比较预测值与实际值
correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax 返回沿着某个维度最大值的位置

#准确率
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


# pred_one_hot = tf.stack(tf.argmin(prediction, 1), tf.argmax(prediction, 1))
# a = tf.argmin(prediction, 1)
# b = tf.argmax(prediction, 1)
# pred_one_hot = tf.transpose(tf.stack([a, b]))

pred_one_hot = tf.one_hot(tf.argmax(prediction, 1), 2)
# pred_one_hot = tf.one_hot(tf.argmax(prediction, 1), tf.shape(prediction)[0])
print(pred_one_hot)

y_sum = tf.reduce_sum(y, 0)

pred_sum = tf.reduce_sum(pred_one_hot, 0)

pred_int_one_hot = tf.one_hot(tf.argmax(prediction, 1), 2, off_value=2.0)
inter = tf.equal(y, pred_int_one_hot)
print(inter)

int_sum = tf.reduce_sum(tf.cast(inter, tf.int32), 0)

# correct_rej = tf.equal(tf.argmax(y, 1), tf.argmax(pred_one_hot, 1))
# int_rej_num = tf.reduce_sum(tf.cast(correct_rej, tf.int32))

# correct_app = tf.equal(tf.argmin(y, 1), tf.argmin(pred_one_hot, 1))
# int_app_num = tf.reduce_sum(tf.cast(correct_app, tf.int32))

init = tf.global_variables_initializer()

Tensor("one_hot_5:0", shape=(?, 2), dtype=float32)
Tensor("Equal_3:0", shape=(?, 2), dtype=bool)


In [7]:
with tf.Session() as sess:
    sess.run(init)
    
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    
    
    for step in range(121):
        # Retrieve a single instance:
        batch_xs, batch_ys = sess.run([train_features, train_labels])
#         print(batch_xs.shape, batch_ys.shape)
#         print(batch_xs)
    
        sess.run(train, feed_dict={x: batch_xs, y: batch_ys})


        if step % 10 == 0:
            acc_train = sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys})
            
            test_xs, test_ys = sess.run([test_features, test_labels])
            pred, acc_test = sess.run([prediction, accuracy], feed_dict={x:test_xs, y:test_ys})
            print("step:" + str(step) + " train accurancy:" + str(acc_train) + "  test accurancy:" + str(acc_test))
            
#             y_s, p_s, rej_cor, app_cor = sess.run(
#                 [y_sum, pred_sum, int_rej_num, int_app_num], 
#                 feed_dict={x:test_xs, y:test_ys} )

            y_s, p_s, i_s = sess.run(
                [y_sum, pred_sum, int_sum], 
                feed_dict={x:test_xs, y:test_ys} )
                
            print("Label sum:" + str(y_s) + " Prediction sum" + str(p_s) + 
                  " Correct sum:" + str(i_s))
                  
    coord.request_stop()
    coord.join(threads)

step:0 train accurancy:0.635  test accurancy:0.52
Label sum:[86. 14.] Prediction sum[54. 46.] Correct sum:[46  6]
step:10 train accurancy:0.735  test accurancy:0.42
Label sum:[86. 14.] Prediction sum[32. 68.] Correct sum:[30 12]
step:20 train accurancy:0.675  test accurancy:0.22
Label sum:[95.  5.] Prediction sum[19. 81.] Correct sum:[18  4]
step:30 train accurancy:0.785  test accurancy:0.2
Label sum:[94.  6.] Prediction sum[14. 86.] Correct sum:[14  6]
step:40 train accurancy:0.715  test accurancy:0.28
Label sum:[96.  4.] Prediction sum[26. 74.] Correct sum:[25  3]
step:50 train accurancy:0.705  test accurancy:0.4
Label sum:[89. 11.] Prediction sum[31. 69.] Correct sum:[30 10]
step:60 train accurancy:0.69  test accurancy:0.26
Label sum:[97.  3.] Prediction sum[23. 77.] Correct sum:[23  3]
step:70 train accurancy:0.76  test accurancy:0.38
Label sum:[94.  6.] Prediction sum[32. 68.] Correct sum:[32  6]
step:80 train accurancy:0.74  test accurancy:0.32
Label sum:[94.  6.] Prediction sum[